In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
torch.__version__
torch.cuda.is_available()

True

In [3]:
from torchvision.datasets import ImageFolder
import torchvision.transforms.v2 as v2

test_transforms = v2.Compose([
    v2.Resize((224, 224)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = ImageFolder('./rose_v1', transform=test_transforms)


c:\Users\user\anaconda3\envs\py3_11\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [4]:
from torch.utils.data import random_split
dataset_train, dataset_test = random_split(dataset, [400, 100])
len(dataset_train), len(dataset_test)

from torch.utils.data import DataLoader
dataloader_train = DataLoader(dataset_train, batch_size=32)
dataloader_test = DataLoader(dataset_test, batch_size=32)

from torchvision.models import resnet50
model = resnet50(weights="IMAGENET1K_V2")
for p in model.parameters():
 p.requires_grad = False


import torch.nn as nn
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(2048, 64),
    nn.ReLU(),
    nn.Linear(64, 2)
)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\user/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100.0%


In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
device

from torch.optim import Adam
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()


dataloader_dict = {
    'train': dataloader_train,
    'val': dataloader_test
}

In [11]:

import time
from tqdm import tqdm

def train_model(model, dataloader_dict, criterion, optimizer, num_epoch):    
    since = time.time()
    best_acc = 0.0
   
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch + 1, num_epoch))
        print('-'*20)
       
        for phase in ['train', 'val']:          
            if phase == 'train':
                model.train()
            else:
                model.eval()
               
            epoch_loss = 0.0
            epoch_corrects = 0
           
            for inputs, labels in tqdm(dataloader_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
               
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                   
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                       
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
                   
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
           
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
           
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
               
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    return model

In [12]:
EPOCHES = 3
train_model(model, dataloader_dict, loss_fn, optimizer, EPOCHES)

Epoch 1/3
--------------------


  0%|          | 0/13 [00:03<?, ?it/s]


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor